<a href="https://colab.research.google.com/github/aman06012003/Emotion-detect/blob/main/Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [3]:
df = pd.read_csv('Emotion_final.csv')

In [4]:
df

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [6]:
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #leave only characters from a to z
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #trun list into sentences
    return " ".join(review)

In [7]:
import nltk
# nltk.download('stopwords')

In [8]:
df['Text']=df['Text'].apply(lambda x: preprocess(x))

In [9]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['N_emotion'] = label_encoder.fit_transform(df['Emotion'])

In [17]:
df['N_emotion'].unique()

array([4, 0, 3, 5, 1, 2])

In [19]:
# Sadness = 4
#anger = 0
#love = 3
# fear=1
# surprise = 5
# happy = 2


In [ ]:
df.head(20)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))#example: the course was long-> [the,the course,the course was,course, course was, course was long,...]

df_cv = cv.fit_transform(df['Text']).toarray()

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test =train_test_split(df_cv, df['N_emotion'], test_size=0.25, random_state=42)

In [ ]:
y_test

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
history= model.fit(X_train, y_train, epochs=10, batch_size=10,validation_data = (X_test,y_test))
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test) 
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
1610/1610 [==============================] - 7s 3ms/step - loss: 0.9804 - accuracy: 0.6583 - val_loss: 0.4633 - val_accuracy: 0.8442
Epoch 2/10
1610/1610 [==============================] - 6s 3ms/step - loss: 0.3001 - accuracy: 0.9012 - val_loss: 0.3860 - val_accuracy: 0.8643
Epoch 3/10
1610/1610 [==============================] - 6s 4ms/step - loss: 0.1652 - accuracy: 0.9464 - val_loss: 0.4052 - val_accuracy: 0.8664
Epoch 4/10
1610/1610 [==============================] - 6s 3ms/step - loss: 0.1106 - accuracy: 0.9657 - val_loss: 0.4711 - val_accuracy: 0.8557
Epoch 5/10
1610/1610 [==============================] - 5s 3ms/step - loss: 0.0787 - accuracy: 0.9755 - val_loss: 0.5228 - val_accuracy: 0.8539
Epoch 6/10
1610/1610 [==============================] - 6s 4ms/step - loss: 0.0571 - accuracy: 0.9817 - val_loss: 0.6027 - val_accuracy: 0.8513
Epoch 7/10
1610/1610 [==============================] - 6s 3ms/step - loss: 0.0458 - accuracy: 0.9858 - val_loss: 0.6737 - val_accuracy:

In [25]:
text='You are smart as fuck'
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = model.predict(array)
a=np.argmax(pred, axis=1)
label_encoder.inverse_transform(a)[0]

1/1 [==============================] - 0s 25ms/step


'happy'

In [23]:
import tensorflow as tf 

In [26]:
model.save('emt_model.h5')

In [27]:
loaded_model = tf.keras.models.load_model('emt_model.h5')

In [32]:
text='You are smart as fuck'
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = loaded_model.predict(array)
a=np.argmax(pred, axis=1)
label_encoder.inverse_transform(a)[0]

1/1 [==============================] - 0s 31ms/step


'happy'

In [31]:
a[0]

2

In [33]:
vocabulary = cv.get_feature_names_out()

# Save the vocabulary to a file
np.savetxt('vocabulary.txt', vocabulary, fmt='%s')